# Analysis
Load results files and run analysis on them

# Functions

## Plotting

In [ ]:
import matplotlib.pyplot as plt
from tigramite import plotting as tp
import numpy as np

def plot_links(results,
               save_name = None,
               figsize = (16, 16),
               node_size = 0.15
              ):
    """
    This function is copied from the basic tutorial, but it may not be
    generalizable: It had issues with output from pcmciplus
    """

    link_matrix = results["link_matrix"]
    var_names = np.array(results["var_names"])
#     val_matrix = results['val_matrix']
    
    links = results["links"]
    linked_variables = set() # Avoids duplicates and sorts when list
    for child, parents_list in links.items():
        if len(parents_list) > 0:
            linked_variables.add(child)
            for parent in parents_list:
                linked_variables.add(parent[0])
    if len(linked_variables) != 0:
        linked_variables = list(linked_variables)
        # Raise an exception if there is a different number of links
        # on the reduced link_matrix
        assert((link_matrix == True).sum()
               == (link_matrix[linked_variables][:,linked_variables] == True).sum())
    else:
        linked_variables = range(len(links))
    
    tp.plot_graph(
        figsize = figsize,
#         val_matrix = val_matrix[linked_variables][:,linked_variables],
        link_matrix = link_matrix[linked_variables][:,linked_variables],
        var_names = var_names[linked_variables],
        link_colorbar_label = 'cross-MCI',
        node_colorbar_label = 'auto-MCI',
        arrow_linewidth = 5,
        node_size = node_size,
        save_name = save_name,
        show_colorbar = False
    ); plt.show()

# Main program

## Specifications

In [ ]:
import sys
import time
import datetime
from pathlib import Path

import utils.utils as utils

from utils.setup import Setup

argv           = sys.argv[1:]
# argv           = ['-c', 'cfg_pipeline.yml']
# argv           = ['-c', 'cfg_concat_lon120.yml']
# argv           = ['-c', 'cfg_single_lon120.yml']

setup = Setup(argv)

Path(setup.plots_folder).mkdir(parents=True, exist_ok=True)


## Aggregate results

In [ ]:
def get_parents_from_links(links):
    linked_variables = set() # Avoids duplicates and sorts when list
    for parents_list in links.values():
        if len(parents_list) > 0:
            linked_variables.add(child)
            for parent in parents_list:
                linked_variables.add(parent[0])
    return [(i in linked_variables)  for i in range(len(links))]


In [ ]:
def aggregate_results_single(key, aggregated_results, setup, errors):
    for i_grid, (lat, lon) in enumerate(setup.gridpoints):
        aggregated_pc_alpha = aggregated_results.get(key, dict())
        results_file = utils.generate_results_filename_single(
                child, level[1], lat, lon, setup.experiment,
                setup.output_file_pattern, setup.output_folder)
        if not results_file.is_file():
#             print(f"Results file {results_file} not found, skipping.")
            not_found_list = errors.get("not_found", list())
            if results_file not in not_found_list:
                not_found_list.append(results_file)
                errors["not_found"] = not_found_list
            continue
        results = utils.load_results(results_file)
        for pc_alpha, alpha_result in results.items():
            if len(alpha_result) > 0:
                links = alpha_result["links"]
                parents = get_parents_from_links(links)
                aggregated = aggregated_pc_alpha.get(pc_alpha, list())
                aggregated.append(parents)
                aggregated_pc_alpha[pc_alpha] = aggregated
                global var_names
                var_names = alpha_result["var_names"] # TODO Very uncomfortable way to obtain this data. Should be metadata
            else:
#                 print(f"File {results_file} is empty, skipping.") # Prints too often
                is_empty_list = errors.get("is_empty", list())
                if results_file not in is_empty_list:
                    is_empty_list.append(results_file)
                    errors["is_empty"] = is_empty_list
        aggregated_results[key] = aggregated_pc_alpha
    # Doesn't return, instead modifies the received `aggregated results` object

    
def aggregate_results_concat(key, aggregated_results, setup, errors):
    aggregated_pc_alpha = aggregated_results.get(key, dict())
    results_file = utils.generate_results_filename_concat(
            child, level[-1], setup.gridpoints, setup.experiment,
            setup.output_file_pattern, setup.output_folder)
    if not results_file.is_file():
#         print(f"Results file {results_file} not found, skipping.")
        not_found_list = errors.get("not_found", list())
        if results_file not in not_found_list:
            not_found_list.append(results_file)
            errors["not_found"] = not_found_list
        return
    results = utils.load_results(results_file)
    for pc_alpha, alpha_result in results.items():
        if len(alpha_result) > 0:
            links = alpha_result["links"]
            parents = get_parents_from_links(links)
            aggregated = aggregated_pc_alpha.get(pc_alpha, list())
            aggregated.append(parents)
            aggregated_pc_alpha[pc_alpha] = aggregated
            global var_names
            var_names = alpha_result["var_names"] # TODO Very uncomfortable way to obtain this data. Should be metadata
        else:
#             print(f"File {results_file} is empty, skipping.") # Prints too often
            is_empty_list = errors.get("is_empty", list())
            if results_file not in is_empty_list:
                is_empty_list.append(results_file)
                errors["is_empty"] = is_empty_list
    aggregated_results[key] = aggregated_pc_alpha
    # Doesn't return, instead modifies the received `aggregated results` object


## Load aggregated results

In [ ]:
def count_total_variables(var_children, levels):
    total = 0
    for child in var_children:
        if child.dimensions == 2:
            total += 1
        elif child.dimensions == 3:
            total += len(levels)
    return total


In [ ]:
import numpy as np

KEY_PATTERN = "{var_name}-{level}"

aggregated_results = dict()
errors = dict()

total_vars = count_total_variables(
    setup.var_children, setup.children_idx_levs)
total_files = total_vars * len(setup.gridpoints)
file_progress = 0
step_progress = 1
step = 5

for child in setup.var_children:
    print(f"Variable: {child.name}")
    
    if child.dimensions == 2:
        child_levels = [[setup.levels[-1],0]]
        key = child.name
    elif child.dimensions == 3:
        child_levels = setup.children_idx_levs
    for level in child_levels:      
        if child.dimensions == 3:
            key = KEY_PATTERN.format(
                    var_name = child.name,
                    level = round(level[0], 2)
            )
        if setup.analysis == "single":
            aggregate_results_single(
                key, aggregated_results, setup, errors)
        elif setup.analysis == "concat":
            aggregate_results_concat(
                key, aggregated_results, setup, errors)
        
        file_progress += len(setup.gridpoints)
        if(file_progress == total_files or
           ((file_progress / total_files * 100) >= step * step_progress)):
            step_progress += 1
            print("Progress: {:.2f}% - {} of {} files".format(
                    file_progress / total_files * 100,
                    file_progress,
                    total_files
                ))


## Print errors found loading aggregated results

In [ ]:
print("ERRORS\n======")
for error_type, error_list in errors.items():
    msg = "{}: {} of {} files ({:.2f}%)".format(
            error_type,
            len(error_list),
            total_files,
            len(error_list)/total_files*100
    )
    print(msg)
    print("-" * len(msg))
    for file in error_list:
        print(file)


## Analyze aggregated results

In [ ]:
# Configuration

# thresholds = [.2, .3, .5, .6]
thresholds = [.3, .5]
# thresholds = [1]
pc_alphas_filter = [str(a) for a in [.001, .01, .1]]
var_names_np = np.array(var_names)

# Print analysis
dict_child_parents = dict()
for child, result in aggregated_results.items():
    print(f"\n{child}")
    dict_pc_alpha_parents = dict()
    for pc_alpha, parents_matrix in result.items():
        dict_threshold_parents = dict()
        if pc_alpha not in pc_alphas_filter:
            continue # Skip this pc alpha
        parents_matrix = np.array(parents_matrix)
        parents_percent = parents_matrix.sum(axis = 0) / parents_matrix.shape[0]
#         dict_pc_alpha_parents["percent"] = parents_percent
        print(f"pc_alpha = {pc_alpha}")
        for threshold in thresholds:
            parents_filtered = parents_percent >= threshold
            parents = [i for i in range(len(parents_filtered)) if parents_filtered[i]]
#             print(parents_filtered)
#             print(parents)
            dict_threshold_parents[str(threshold)] = parents
            print(f"* Threshold {threshold}:\t{var_names_np[parents]}")
        dict_pc_alpha_parents[pc_alpha] = dict_threshold_parents
    dict_child_parents[child] = dict_pc_alpha_parents


## Plots

### Plot for each pc_alpha and threshold combination with all children

#### Generate results file for filter combination

In [ ]:
var_names_parents = var_names[:-1] # NOTE: This assumes that the list has only one child
var_names_children = list()

dict_combinations = dict()
len_parents = len(var_names_parents)
len_total = len_parents + len(dict_child_parents)
for i_child, (child, dict_pc_alpha_parents) in enumerate(dict_child_parents.items()):
    i_child = i_child + len_parents
    var_names_children.append(child)
    for pc_alpha, dict_threshold_parents in dict_pc_alpha_parents.items():
        for threshold, parents in dict_threshold_parents.items():
            key = f"a{pc_alpha}-t{threshold}"
            combination_results = dict_combinations.get(key, dict())
            
            # Build links
            links = combination_results.get(
                    "links", {i : [] for i in range(len(var_names_parents))})
#             links[i_child] = [(parent, -1) for parent in parents]
            links[i_child] = [(parent, 0) for parent in parents]
            combination_results["links"] = links
            
            # Build link_matrix
            link_matrix = combination_results.get(
                    "link_matrix",
#                     [[[False, False] for i in range(len_total)]
                    [[[False] for i in range(len_total)]
                         for j in range(len_total)])
            for parent in parents:
#                 link_matrix[parent][i_child] = [False, True]
                link_matrix[parent][i_child] = [True]
            combination_results["link_matrix"] = link_matrix
            
            # Store results
            dict_combinations[key] = combination_results

all_var_names = var_names_parents + var_names_children # Concatenation
for combination, combination_results in dict_combinations.items():
    combination_results["link_matrix"] = np.array(combination_results["link_matrix"])
    combination_results["var_names"] = all_var_names


#### Print plots

In [ ]:
for combination, combination_results in dict_combinations.items():
    print(combination)
    plot_filename = "{cfg}_{combination}.png".format(
            cfg = setup.yml_filename.rsplit(".")[0],
            combination = combination
    )
    plot_file = Path(setup.plots_folder, plot_filename)
    if not setup.overwrite and plot_file.is_file():
        print(f"Found file {plot_file}, skipping.")
        continue # Ignore this result
    plot_links(
        combination_results,
        save_name = plot_file,
        figsize = (48, 48),
        node_size = 0.05
    )


### Plot every pc alpha of a single file

In [ ]:
# Load data
len_grid = len(setup.gridpoints)
t_start = time.time()
for i_grid, (lat, lon) in enumerate(setup.gridpoints):
    if setup.analysis == "concat":
        print("Analysis of concatenated files is currently unsupported, skipping.")
        break
    idx_lat = setup.idx_lats[i_grid]
    idx_lon = setup.idx_lons[i_grid]
    
    t_start_gridpoint = time.time()
    print(f"Gridpoint {i_grid+1}/{len_grid}:"
          + f"lat={lat} ({idx_lat}), lon={lon} ({idx_lon})")
    for child in setup.var_children:
        print(f"Variable: {child.name}")
        if child.dimensions == 2:
            child_levels = [[setup.levels[-1],0]]
        elif child.dimensions == 3:
            child_levels = setup.children_idx_levs
        for level in child_levels:
            results_file = utils.generate_results_filename_single(
                    child, level[1], lat, lon, setup.experiment,
                    setup.output_file_pattern, setup.output_folder)
            if not results_file.is_file():
                print(f"Results file {results_file} not found, skipping.")
                continue
            results = utils.load_results(results_file)

            print(f"Plotting links for {child.name} at level {level[1]+1}")
            t_before_plot_linkgs = time.time()
            # Plotting
            for pc_alpha, alpha_result in results.items():
                print(f"pc_alpha = {pc_alpha}")
                plot_file = Path(
                        setup.plots_folder,
                        setup.plot_file_pattern.format(
                            var_name = child.name,
                            level = level[1]+1,
                            lat = int(lat),
                            lon = int(lon),
                            pc_alpha = pc_alpha,
                            experiment = setup.experiment
                        )
                )
                if not setup.overwrite and plot_file.is_file():
                    print(f"Found file {plot_file}, skipping.")
                    continue # Ignore this result
                if len(alpha_result) > 0:
                    print(f"Plotting to {plot_file}")
                    plot_links(alpha_result, save_name = plot_file)
                else:
                    print("Results are empty, skipping.")
            time_plot_links = datetime.timedelta(
                    seconds = time.time() - t_before_plot_linkgs
            )
            print(f"Plotted. Time: {time_plot_links}")

    time_point = datetime.timedelta(
            seconds = time.time() - t_start_gridpoint)
    print(f"All links in gridpoint plotted. Time: {time_point}")
total_time = datetime.timedelta(seconds = time.time() - t_start)
print(f"Execution complete. Total time: {total_time}")
    